Загружаем данные о графе

In [1]:
!wget https://snap.stanford.edu/data/amazon0302.txt.gz

connected.
HTTP request sent, awaiting response... 200 OK
Length: 4664334 (4.4M) [application/x-gzip]
Saving to: ‘amazon0302.txt.gz’

amazon0302.txt.gz   100%[===================>]   4.45M  1.76MB/s    in 2.5s    

2024-05-21 14:24:17 (1.76 MB/s) - ‘amazon0302.txt.gz’ saved [4664334/4664334]



In [2]:
!gzip -d ./amazon0302.txt.gz

gzip: ./amazon0302.txt already exists; do you wish to overwrite (y or n)? ^C


In [ ]:
!wget https://snap.stanford.edu/data/congress_network.zip
!unzip congress_network.zip
!rm congress_network.zip

--2024-05-21 15:11:57--  https://snap.stanford.edu/data/congress_network.zip
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80, 198.41.0.4, 170.247.170.2, ...
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 193815 (189K) [application/zip]
Saving to: ‘congress_network.zip’

congress_network.zi 100%[===================>] 189.27K   217KB/s    in 0.9s    

2024-05-21 15:11:59 (217 KB/s) - ‘congress_network.zip’ saved [193815/193815]

Archive:  congress_network.zip
replace congress_network/compute_vc.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!wget https://snap.stanford.edu/data/wiki-Vote.txt.gz
!gzip -d ./wiki-Vote.txt.gz

In [ ]:
!git clone https://github.com/OafishSpark/InfluencerSearchAlgorithms

!mv /content/InfluencerSearchAlgorithms/* /content/
!rm -rf /content/InfluencerSearchAlgorithms

In [ ]:
import networkx as nx

import numpy as np
from numpy import random

from tqdm import tqdm

In [ ]:
from models.threshold_model import linear_threshold_model
from models.sir_model import sir_model, sir_inf_model

from influencers.greedy_kkt import greedy_kkt_influencers
from influencers.centrality import centrality_influencers
from influencers.shapley_value import shapley_value_influencers
from influencers.csr import csr_influencers
from influencers.hierarchy import hierarchy_influencers_dull, hierarchy_influencers_with_csr, hierarchy_influencers_with_centrality

from communities.clustering import clustering_communities
from communities.shapley_value import shapley_value_communities

from communities.hierarchy import hierarchy_communities_original, hierarchy_communities_improved

In [ ]:
graph_congress = nx.read_edgelist("./congress_network/congress.edgelist", create_using=nx.DiGraph())

In [ ]:
graph_wiki_votes = nx.read_edgelist("./wiki-Vote.txt", create_using=nx.DiGraph())
nx.set_edge_attributes(graph_wiki_votes, 1, 'weight')

In [ ]:
subgraph_wiki_votes = nx.subgraph(graph_wiki_votes, list(graph_wiki_votes.nodes)[1530:1730])

In [ ]:
# graph_epinions = nx.read_edgelist("./wiki-Talk.txt", create_using=nx.DiGraph())
# nx.set_edge_attributes(graph_epinions, 1, 'weight')

In [ ]:
# subgraph_epinions = nx.subgraph(graph_epinions, list(graph_epinions.nodes)[0:100])

In [ ]:
graphs = {
    "graph congress": graph_congress,
    "graph wiki votes": subgraph_wiki_votes,
    # "graph epinions": subgraph_epinions,
}

In [ ]:
for name in graphs.keys():
  print(name)
  graph = graphs[name]
  print(graph.number_of_nodes())
  print(graph.number_of_edges())
  print(nx.density(graph))
  print(nx.number_strongly_connected_components(graph))
  print("")

In [ ]:
graph = graphs["graph congress"]
# graph = graphs["graph epinions"]
# graph = graphs["graph wiki votes"]

Communities

In [ ]:
communities = {
    "communities shapley": shapley_value_communities(graph, 5),
    "communities clusters": clustering_communities(graph, 5),
    "communities hierarchy original": hierarchy_communities_original(graph, 5),
    "communities hierarchy improved": hierarchy_communities_improved(graph, 5)
}

Influencers

In [ ]:
influencers_pack = {
    "influencers hierarchy with centrality": hierarchy_influencers_with_centrality(graph, 5),
    "influencers shapley": shapley_value_influencers(graph, 5),
    "influencers centrality": centrality_influencers(graph, 5),
    "influencers kkt": greedy_kkt_influencers(graph, 5),
    "influencers hierarchy orig": hierarchy_influencers_dull(graph, 5),
    "influencers hierarchy improved": hierarchy_influencers_with_csr(graph, 5),
}

In [ ]:
influencers_csr = dict(zip(
    ['csr ' + name for name in communities.keys()],
    [csr_influencers(graph, 5, community) for community in communities.values()]
))

In [ ]:
influencers_pack.update(influencers_csr)

In [ ]:
def perform_experiment(
    n_experiments: int,
    graph: nx.DiGraph,
    influencers_dict: dict,
    time: int = 5,
) -> dict:
  nodes = list(graph.nodes)
  n_nodes = len(nodes)
  results = dict(zip(
    ["linear threshold model", "SIR model", "Infinite SIR model"],
    [dict([(name, 0) for name in influencers_dict.keys()]) for _ in range(3)]
  ))
  for _ in tqdm(range(n_experiments)):
    theta = dict(zip(list(graph.nodes), random.uniform(0.01, 0.99, graph.number_of_nodes())))
    # infection_prob = dict(zip(nodes, random.uniform(0.01, 0.99, n_nodes)))
    # recover_prob = dict(zip(nodes, random.uniform(0.01, 0.99, n_nodes)))
    for name in influencers_dict.keys():
      influencers = influencers_dict[name]
      results['linear threshold model'][name] += len(linear_threshold_model(graph, influencers, time, theta)) / n_experiments
      results['SIR model'][name] += len(sir_model(graph, influencers, time)) / n_experiments
  for name in influencers_dict.keys():
    influencers = influencers_dict[name]
    results['Infinite SIR model'][name] += len(sir_inf_model(graph, influencers, time))
  return results

In [ ]:
number_of_experiments = 100

In [ ]:
times = [4*i + 1 for i in range(10)]
results = dict(zip(
    times,
    [perform_experiment(number_of_experiments, graph, influencers_pack, time) for time in times]
))

In [ ]:
times

In [ ]:
results

In [ ]:
for results_t in results.values():
    for model in results_t.keys():
      print(model + ":")
      for infl_alg in sorted(results_t[model], reverse=True, key=lambda x: results_t[model][x])[:5]:
        print(infl_alg + ": " + str(results_t[model][infl_alg]))
      print('\n')

In [ ]:
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
inner_lists = {
    "influencers hierarchy with centrality": [],
    "influencers kkt" : [],
    "influencers shapley": [],
    "influencers centrality": [],
    "influencers kkt": [],
    "influencers hierarchy orig": [],
    "influencers hierarchy improved": [],
    "csr communities shapley": [],
    "csr communities clusters": [],
    "csr communities hierarchy original": [],
    "csr communities hierarchy improved": [],
}

results_lists = {
    "linear threshold model": inner_lists,
    "SIR model": deepcopy(inner_lists),
    "Infinite SIR model": deepcopy(inner_lists),
}

for t in times:
    for model in results[t].keys():
        for key in results[t][model].keys():
            results_lists[model][key].append(results[t][model][key])

In [ ]:
results_lists['linear threshold model']['influencers hierarchy with centrality']

In [ ]:
fig, ax = plt.subplots()

for key in results_lists["linear threshold model"]:
    if key != "influencers hierarchy improved":
        ax.plot(times, results_lists["linear threshold model"][key], label = key)

plt.legend(loc='upper left')

plt.show()

In [ ]:
fig, ax = plt.subplots()

for key in results_lists["SIR model"]:
    if key != "influencers hierarchy improved":
        ax.plot(times, results_lists["SIR model"][key], label = key)

plt.legend(loc='upper left')

plt.show()

In [ ]:
nx.draw(graph)

In [ ]:
for community in communities.keys(): 
    print(community, nx.community.modularity(graph, communities[community]))

In [ ]:
for community in communities.keys(): 
    print(community, nx.community.partition_quality(graph, communities[community]))

In [ ]:
nx_communities = nx.community.greedy_modularity_communities(graph, 10)
print(nx.community.modularity(graph, nx_communities))
print(nx.community.partition_quality(graph, nx_communities))

In [ ]:
edges = list(graph.edges)
edges[0]

In [ ]:
activating = {'a': 1, 'b': 0, 'c': 0, 'd': 2, 'e': 3, 'f': 3}
activated = [node for node in filter(lambda x: activating[x] > 0, activating.keys())]
activated

In [ ]:
b = 0
for a in graph.in_edges('0'):
    b = graph.edges[a]['weight']
b

In [ ]:
nx.transitive_closure(graph)

In [ ]:
nx.adjacency_matrix(graph).todense()

In [ ]:
a = np.array([0, 0, 0])
b = np.array([[1, 0, 0],[1, 0, 0],[0, 0, 1]])
np.sum(np.logical_or(a, b[1]).astype(int))